## Approximating Costs

In [ ]:
import pandas as pd

In [ ]:
results_df_pandas = pd.read_csv('AWS Results/results_df_pandas.csv',index_col=0)
results_df_pandas['Cost'] = results_df_pandas['Average']/3600 * 0.2

In [ ]:
results_df_rapids = pd.read_csv('AWS Results/results_df_rapids_etl.csv',index_col=0)
results_df_rapids['Cost'] = results_df_rapids['Average']/3600 * 0.379

In [ ]:
spark = pd.read_csv('AWS Results/spark_etl_results.csv')

In [ ]:
spark['Cost'] = spark['Average']/3600*(3*0.2)

## Vizualization

In [ ]:
import os
os.environ[ 'MPLCONFIGDIR' ] = '/tmp/'
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Defining the plot that is used
def plot(axs, averages_pandas, averages_rapids, averages_spark, test):
    axs.bar(x = ind, 
            height=averages_pandas, 
            width = width, 
            color='royalblue', label='Pandas')

    axs.bar(ind+width, 
                height=averages_rapids, 
                width = width, 
                color='seagreen',label='Rapids')
    
    axs.bar(ind+width+width, 
        height=averages_spark, 
        width = width, 
        color='purple',label='Spark')

    axs.set_ylabel('Average Cost ($)')
    axs.set_xlabel('Data Set Size (GB)')
    axs.xaxis.label.set_fontsize(20)
    axs.yaxis.label.set_fontsize(20)
    for item in (axs.get_xticklabels() + axs.get_yticklabels()):
        item.set_fontsize(17)
        
    axs.set_title(f"Test: {test}",fontweight="bold", fontsize=25)
    axs.legend(loc='upper left', prop={'size': 25})
    axs.set_xticks(ticks=ind + width/2, 
              labels=bar_labels)
    
    if test == 'Mean':
        ax_new = subfigs[0].add_axes([0.238, 0.785, 0.25, 0.17])
        plt.ylim(0, 0.0000002)
    elif test == 'Sort':
        ax_new = subfigs[0].add_axes([0.743, 0.785, 0.25, 0.17])
        plt.ylim(0, 0.000008)
    elif test == 'Merge':
        ax_new = subfigs[0].add_axes([0.238, 0.280, 0.25, 0.17])
        plt.ylim(0, 0.00002)
    elif test == 'Filter':
        ax_new = subfigs[0].add_axes([0.743, 0.280, 0.25, 0.17])
        plt.ylim(0, 0.000002)
    else:
        ax_new = subfigs[1].add_axes([0.693, 0.565, 0.3, 0.35])
        plt.ylim(0, 0.00004)
        
        
    ax_new.bar(x = ind, 
            height=averages_pandas, 
            width = width, 
            color='royalblue', label='Scikit-Learn')

    ax_new.bar(ind+width, 
                height=averages_rapids, 
                width = width, 
                color='seagreen',label='Rapids')

    ax_new.bar(ind+width+width, 
            height=averages_spark, 
            width = width, 
            color='purple',label='Spark')

    ax_new.set_xticks([])
    return axs

In [ ]:
# Creating the plot matrix

NUM_DSIZE_DOUBLINGS = 12

# Computing an array to hold the test names
tests = np.append(results_df_pandas['Test'].drop_duplicates().values,'None').reshape((3,2))

# Defining the figure
fig = plt.figure(constrained_layout=True,figsize=(3,3))
fig.tight_layout(pad=5.0)

fig.suptitle('Pandas vs Rapids vs Spark Cost Comparison', fontsize=30, fontweight="bold")

# Setting the fig size
fig.set_size_inches(25, 20)

# Creating the subfigures
subfigs = fig.subfigures(2, 1, wspace=1,height_ratios=[2, 1.])
axs = subfigs[0].subplots(2, 2)

# Setting the number of bars
ind = np.arange(0, NUM_DSIZE_DOUBLINGS*2, 2)

# Defining the width between the bars
width = 0.55

# Defining the bar labels
bar_labels = np.round(results_df_pandas['Test Size'].drop_duplicates(), 3).astype(str)

# Creating the plots
for i in range(0, 2):
    for j in range(0, 2):
        test = tests[i, j]            
        plot(axs[i,j], 
             results_df_pandas[results_df_pandas['Test'] == test]['Cost'],
             results_df_rapids[results_df_rapids['Test'] == test]['Cost'],
             spark[spark['Test'] == test]['Cost'],
             test)
# Creating the bottom plot since there is an odd number
ax = subfigs[1].subplots(1,1)
plot(ax, 
     results_df_pandas[results_df_pandas['Test'] == 'All']['Cost'],
     results_df_rapids[results_df_rapids['Test'] == 'All']['Cost'],
     spark[spark['Test'] == 'All']['Cost'],
     'All')
plt.savefig('AWS Results/etl_cost.png')
plt.show()